# Talk to your App

In majority of the examples, LLM Usage is shown as a chat-bot or personal assistant. 

A more powerful usage of Agentic-AI are when they interact with the system in the background and enrich it or take autonomous actions. This is conceptually similar to streaming analytics - with analytics being processed by Agents powered by LLMs.

You will see that for chat-bot type usage - if the AI infrastructure is down, things still work.
However for the Agents in backend - if the AI system is down, then the system will suffer some down time.

_Each module is typically dependent on the prior modules having been completed successfully_

### Verifying Python Software Installation

The following `pip install` command ensures all necessary Python software components (dependencies) are present. 
It's safe to run for verification, even if you've already completed Module 02 (which should have handled this).

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner,AsyncOpenAI,OpenAIChatCompletionsModel
from dotenv import load_dotenv, find_dotenv
from rich.pretty import pprint
from rich import print

api_key = "sk-dummy_key"
os.environ["OPENAI_API_KEY"] = api_key

model = "llama3.2:3b-instruct-fp16" 
base_url = "http://localhost:11434/v1/"

model = OpenAIChatCompletionsModel( 
    model=model,
    openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key=api_key),
)

print("[green] Model setup[/green]")

 Model setup

## Automated Error Handling

1.The process is started when an ansible job log completion data comes in. (as of now, we simulate this as a user input)
1. It examimes if there is any error. If no error, it ends
1. If there is an error:
    - Agent analyzes and recommends
    - Agent opens a jira ticket
    - Agent sends a slack message

![Workflow](resources/images/agent_log_workflow.png)

In [3]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace


@dataclass
class Advisor:
    body: str

@dataclass
class Slacker:
    body: str

@dataclass
class JIRAer:
    body: str


advisor_agent = Agent(
    name="advisor_agent",
    instructions=(
        "You can look at the contents of an ansible log and spot the error.\n"
        "You will describe what the error is in a few crisp sentences so that a human can take corrective actions.\n"
    ),
    model = model,
    output_type=Advisor,
    model_settings=ModelSettings(temperature=0.0),
)

slack_agent = Agent(
    name="slack_agent",
    instructions=(
        "If there is an error captured in the input message, you will always output - I have slacked the error message.\n"
        "Also add the contents that is the provided in the input. \n"
        "Give a made up slack link. \n"
        "If there is no error, then just say - All is well, there is nothing to be done.\n"
    ),
    model = model,
    output_type=Slacker,
    model_settings=ModelSettings(temperature=0.0),
)

jira_agent = Agent(
    name="jira_agent",
    instructions=(
        "If there is an error captured in the input message, you will always output - I have created a JIRA issue.\n"
        "Also add the contents that is the provided in the input. \n"
        "Give a made up slack link. \n"
        "If there is no error, then just say - All is well, there is nothing to be done.\n"
    ),
    model = model,
    output_type=JIRAer,
    model_settings=ModelSettings(temperature=0.0),
)

print("[green] Agents setup[/green]")

 Agents setup

In [4]:
#msg = "all tasks successfully finished"
msg = "could not connect to the host as the password as expired. "
inputs = [{"content": msg, "role": "user"}]

#with trace("Router"):
#    story_outline_result = await Runner.run(advisor_agent,inputs)
#    #uncomment this to see the details
#    pprint(story_outline_result)
#    print("--------------------------")
#    print(story_outline_result.final_output)

with trace("Workflow"):
    print("----------Advisor Output----------")
    advisor_result = await Runner.run(advisor_agent,inputs)
    print(advisor_result.final_output.body)
    advisor_output: Advisor = advisor_result.final_output
    
    print("----------JIRA Output----------")
    jira_result = await Runner.run(jira_agent,advisor_output.body)
    print(jira_result.final_output.body)
    
    print("----------Slack Output----------")
    slack_result = await Runner.run(slack_agent,advisor_output.body)
    print(slack_result.final_output.body)

----------Advisor Output----------

Connection refused

Last failure occurred 1 minute and 30 seconds ago

Failed to connect to host: Password has expired

Last error: Password has expired

Original exception: <AnsibleError: 'Password has expired'
> at AnsibleInventoryItem._get_password_from_file(self, inventory_file)
 at AnsibleInventoryItem.__init__(self, inventory_file, group_by, host_list)
 at AnsibleInventoryItem.__init__(self, inventory_file, group_by, host_list)
 at AnsibleInventoryFile.parse_inventory_file(self, inventory_file)
 at AnsibleInventoryFile._parse_host_group(self, host_group)
 at AnsibleInventoryFile._parse_host_group(self, host_group)
 at AnsibleInventoryFile.parse_inventory_file(self, inventory_file)

----------JIRA Output----------

<html><body>Connection refused</body></html>

----------Slack Output----------

<html><body> <h1>Slack Error</h1> <p> I have slack'd the error message: Password has expired. </p> <p> The contents
provided in the input were:</p> <pre><code>Connection refused<br>Last failure occurred 1 minute and 30 seconds 
ago<br>Failed to connect to host: Password has expired<br>Last error: Password has expired<br>Original exception: 
<AnsibleError: 'Password has expired' at AnsibleInventoryItem._get_password_from_file(self, inventory_file) at 
AnsibleInventoryItem.__init__(self, inventory_file, group_by, host_list) at AnsibleInventoryItem.__init__(self, 
inventory_file, group_by, host_list) at AnsibleInventoryFile.parse_inventory_file(self, inventory_file) at 
AnsibleInventoryFile._parse_host_group(self, host_group) at AnsibleInventoryFile._parse_host_group(self, 
host_group) at AnsibleInventoryFile.parse_inventory_file(self, inventory_file)</code></pre> <p> For more 
information, visit this link: https://example.com/slack-error </p> </body></html>

# Human-in-the-Loop
But LLMs hallucinate!

Yes, LLMs, just like traditional AI and human beings may not always give the right answer. To handle those kind of possible mistakes, we have processes in place.

In the above example, we are seamlessly blending human-in-the-loop when we open a JIRA ticket or Slack a message. Even if the contents of these are not entirely accurate, we do not lose much because a person can check and correct if needed. 

In distributed systems, there are lots of similar examples - which has been around us for a long time - to take care of possible errors: for example those which arise out of CAP Theorem related inconsistencies. 


## Extend the Debugging Agent capabilities to make it robust
We shared in 05-agents.ipynb a debugger agent. Let us make it more robust. What happens if the answer given by the debugger has obvious gaps or hallucinates. Can we get another agent to review it and fix it? Let us see.

### This is a clone of the last lesson

In [5]:
@function_tool
def get_dependency(service:str) ->list[str]:
    dep_service=["ProductCatalogService","CheckoutService","UserProfileService"] 
    return dep_service

did_agent = Agent(
    name="DependencyIdentifier Agent",
    instructions=(
        "An incident will be passed on.\n"
        "From that, firstly identify the affected service name only.\n"
        "Next, identify what are the service dependencies for that service.\n"
        "Just return all service names in a comma separated format like a python list[str]. Also include the affected service.\n"
        "And nothing else"
    ),
    model=model,
    tools=[get_dependency],
    model_settings=ModelSettings(temperature=0.0),
)

@function_tool
def get_changelog(service:list) ->list[str]:
    change_log=["ProductCatalogService changed","CheckoutService changed"]
    return change_log

change_agent = Agent(
    name="ChangeLog Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..'].\n"
        "Identify what has changed with these services and return them.\n"
        "Use available tools for this and do not generate data on our own.\n"
        "Just return all changes in a comma separated format like a python list[str].\n"
        "Do not return duplicate changes"
    ),
    model= model,
    tools=[get_changelog],
    model_settings=ModelSettings(temperature=0.0),
)

@function_tool
def get_errorlog(service:list) ->list[str]:
    error_log=["ProductCatalogService is responding slowly"]
    return error_log

error_agent = Agent(
    name="Error Log Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..']. \n"
        "Note that all services may not have error messages and it is unlikely that same message appear in logs of all services. \n"
        "Use available tools for this and do not generate data on our own.\n"
        "The error messages will have service names in the messages. \n"
        "Identify which services have what kind of errors"
    ),
    model=model,
    tools=[get_errorlog],
    model_settings=ModelSettings(temperature=0.0),
)

debugger_agent = Agent(
    name="Debugger Agent",
    instructions=(
        "You will be given:\n"
        "1. Incident details.\n"
        "2. Services that could have been root cause of the problem.\n"
        "3. Services that were changed in the time interval.\n"
        "4. Services that had errors in the logs.\n"
        "Based on the above, loigically think through and conclude the most likely reason for this problem. \n"
        "Please lay down your thought process clearly that led you to the conclusion. "
    ),
    model= model,
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] Agents setup[/green]")

 Agents setup

### Now let us add a new agent
We add a Verification Agent whose sole job is to audit each diagnosis before you act on it. In practice this agent will:

- Read the incident summary, the list of services, and the debugger’s reasoning
- Check for mismatches or missing facts (e.g., a service name dropped or an error overlooked)
- Flag any inconsistencies or confirm “All clear”

By doing so, we get an extra safety net that catches accidental oversights or AI hallucinations. 

In [6]:
verification_agent = Agent(
    name="Verification Agent",
    instructions=(
        "You’ll be given four parts:\n"
        "1) The incident description\n"
        "2) A list of services\n"
        "3) The debugger agent’s full reasoning and conclusion\n\n"
        "Check for any of these issues:\n"
        " • References to services not in the original list\n"
        " • Conclusions that contradict the provided errors/changes\n"
        " • Missing any service that clearly had errors or changes\n\n"
        "If everything is consistent, reply “Consistent”. Otherwise, list the problems."
    ),
    model=model,
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] Verification Agent setup[/green]")

 Verification Agent setup

### Notice the new section added under orchestrate
Invokes the verification agent after debugger agent is done.

In [7]:
import asyncio
async def orchestrate(input):
    # Call the intermediate agents to gather the facts
    # These all use tools heavily
    dep_result = await Runner.run(did_agent,input)
    change_result = await Runner.run(change_agent, dep_result.final_output)
    error_result = await Runner.run(error_agent, dep_result.final_output)

    services = dep_result.final_output               # e.g. ["foo","bar","baz"]
    changes  = change_result.final_output             # e.g. ["foo changed","bar changed"]
    errors   = error_result.final_output              # e.g. ["foo is responding slowly"]

    # Build a single prompt string:
    message = (
        "Incident details: " + input + "\n"
        "Affected services: " + services + "\n"
        "Changes detected: " + changes + "\n"
        "Error logs: " + errors + "\n"
        "Based on the above, logically think through and conclude the most likely reason for this problem. "
        "Please lay down your thought process clearly that led you to the conclusion."
    )
    print("\n")
    print("Input to the Deubgger Agent: ")
    print("-----------------------------")
    print(message)
    print("\n")
    # Invoke it:
    debugger_result = await Runner.run(debugger_agent, message)
    print("=== Debugger Thought Process & Conclusion ===")
    print(debugger_result.final_output)

    # New Section:    
    verification_prompt = (
        "Incident details: " + input + "\n"
        "Affected services: " + services + "\n"
        f"Debugger reasoning:\n{debugger_result.final_output}\n\n"
        "Please check:\n"
        " • Are all referenced services in the original list?\n"
        " • Does the conclusion contradict any errors/changes?\n"
        "If everything is consistent, just say “Consistent” and nothing else. "
        "Otherwise list the issues you see."
    )
    
    print("\n")
    print("Input to the Verification Agent: ")
    print("---------------------------------")
    print(verification_prompt)
    print("\n")

    verification_result = await Runner.run(verification_agent, verification_prompt)
    #pprint(verification_result.final_output)
    return verification_result.final_output

print("[green] Agent orchestration setup[/green]")

 Agent orchestration setup

### Calling the orchestration function as in the past

In [8]:
input = "Incident: ShoppingCart response time has increased to 10 sec"
diagnosis = await orchestrate(input)
print("=============================================")
print("=== Verifier Thought Process & Conclusion ===")
print("=============================================")
print(diagnosis)

Input to the Deubgger Agent:

-----------------------------

Incident details: Incident: ShoppingCart response time has increased to 10 sec
Affected services: The affected service is ShoppingCart. The dependencies for this service are 
ProductCatalogService, CheckoutService, and UserProfileService.
Changes detected: The changes for the affected service ShoppingCart are:

ProductCatalogService changed, 
CheckoutService changed
Error logs: It appears that the ProductCatalogService is experiencing slow response times. This could be causing 
issues with the ShoppingCart service, which relies on it.

The CheckoutService and UserProfileService do not have any error messages in their logs at this time. 

Please note that you may want to investigate further into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue.
Based on the above, logically think through and conclude the most likely reason for this problem. Please lay down 
your thought process clearly that led you to the conclusion.

=== Debugger Thought Process & Conclusion ===

Here's my thought process:

**Step 1: Identify the affected service**
The incident is affecting the ShoppingCart service, which relies on three dependencies: ProductCatalogService, 
CheckoutService, and UserProfileService.

**Step 2: Analyze the changes detected**
Two of the dependencies, ProductCatalogService and CheckoutService, have been changed recently. This suggests that 
these changes might be contributing to the issue with the ShoppingCart service.

**Step 3: Investigate error logs**
The error log indicates that ProductCatalogService is experiencing slow response times, which could be causing 
issues with the ShoppingCart service. This is a strong indication of a potential root cause.

**Step 4: Eliminate unlikely causes**
Although CheckoutService and UserProfileService have been changed recently, there are no error messages in their 
logs at this time. This suggests that they might not be contributing to the issue with the ShoppingCart service.

**Step 5: Draw conclusions**
Based on the analysis, it is likely that the recent changes to ProductCatalogService are causing the slow response 
times and subsequent issues with the ShoppingCart service. The fact that CheckoutService and UserProfileService do 
not have error messages in their logs at this time suggests that they might be unaffected by these changes.

**Conclusion**
The most likely reason for the increased response time of the ShoppingCart service is the recent change to 
ProductCatalogService, which is experiencing slow response times. This change is likely causing a ripple effect, 
impacting the ShoppingCart service and its dependencies.

Recommendation:
To resolve this issue, I would recommend investigating further into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue. Additionally, monitoring the performance of 
the ProductCatalogService and other affected services to prevent similar issues in the future.

...

Incident details: Incident: ShoppingCart response time has increased to 10 sec
Affected services: The affected service is ShoppingCart. The dependencies for this service are 
ProductCatalogService, CheckoutService, and UserProfileService.
Debugger reasoning:
Here's my thought process:

**Step 1: Identify the affected service**
The incident is affecting the ShoppingCart service, which relies on three dependencies: ProductCatalogService, 
CheckoutService, and UserProfileService.

**Step 2: Analyze the changes detected**
Two of the dependencies, ProductCatalogService and CheckoutService, have been changed recently. This suggests that 
these changes might be contributing to the issue with the ShoppingCart service.

**Step 3: Investigate error logs**
The error log indicates that ProductCatalogService is experiencing slow response times, which could be causing 
issues with the ShoppingCart service. This is a strong indication of a potential root cause.

**Step 4: Eliminate unlikely causes**
Although CheckoutService and UserProfileService have been changed recently, there are no error messages in their 
logs at this time. This suggests that they might not be contributing to the issue with the ShoppingCart service.

**Step 5: Draw conclusions**
Based on the analysis, it is likely that the recent changes to ProductCatalogService are causing the slow response 
times and subsequent issues with the ShoppingCart service. The fact that CheckoutService and UserProfileService do 
not have error messages in their logs at this time suggests that they might be unaffected by these changes.

**Conclusion**
The most likely reason for the increased response time of the ShoppingCart service is the recent change to 
ProductCatalogService, which is experiencing slow response times. This change is likely causing a ripple effect, 
impacting the ShoppingCart service and its dependencies.

Recommendation:
To resolve this issue, I would recommend investigating further into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue. Additionally, monitoring the performance of 
the ProductCatalogService and other affected services to prevent similar issues in the future.

Please check:
 • Are all referenced services in the original list?
 • Does the conclusion contradict any errors/changes?
If everything is consistent, just say “Consistent” and nothing else. Otherwise list the issues you see.

=============================================

=== Verifier Thought Process & Conclusion ===

=============================================

Consistent

# Going into Production

We have seen now how a so called output by AI can be cross checked. This is widely used pattern in Agentic workflows.

One other important issue - how does the system Learn ?

1. Let us say Agentic application gives a certain resolution to an incident.
1. And the engineer verifies it to be correct.
1. Or the engineer verifies it not be correct and knows the correct solution.

How can we enhance our agentic application with this.

## Learning
### Integrate Human Feedback
- Whenever the Verification Agent flags a problem, route the case to an engineer for review.
- Provide a simple thumbs-up/thumbs-down or rating interface. Feed that rating back into your store.
- Also think of allowing general text entry field allowing engineer to enter what was done, if it was a thumbs down

### Capture and store outcomes
- Capture the above data and store it in a database

### Feedback the data
- Provide this data as an additional context to the debugger agent (or perhaps add another agent) which looks at this data and fine tunes the recommendation.

### Continuously refine your agents
- Periodically pull the best-rated incident examples (and their human-approved diagnoses) to create few-shot prompts or even fine-tune a custom model.
- Update agent instructions based on common failure modes (e.g. “always double-check inventory data”).

Some of the other next steps could be -

## Build visibility and dashboards
- Surface the agents’ findings and verification results in a team dashboard—showing average time to diagnosis, verification pass rates, and automation success rates.
- Use that data to spot gaps (e.g. Services that consistently fool the Debugger) and add new special-purpose agents.

## Hook into real incident streams
- Connect the orchestration function to the monitoring/alerting system (e.g. Prometheus Alertmanager, CloudWatch Alarms, PagerDuty webhooks). Therefore every time an alert fires for “shopping-cart latency,” the agents automatically kick off the dependency→change→error→debug→verify chain.

## Future State
- Let us imagine a state in future where there may be many agents that can gather data. In the above example it was confined to 3:
  - discovering service dependency,
  - looking at change log and
  - looking at application logs.
Let us say there are agents for metrics, anomaly detection, cluster health (for the cluster on which the service is running on etc).
- In that case, the incident could go to a `Planner agent` that decides to breakdown the troubleshooting into steps and call agents for each steps.
- And then hand over the summary to the Debugger Agent and Verifier agent as shown above.